In [9]:
# Multiple outputs per cell in Jupyter 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%load_ext jupyternotify
%autonotify -a 30

<IPython.core.display.Javascript object>

In [2]:
%%javascript
// Evitar autoscroll.
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [24]:
from integrado import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
tokens=Tokens().tokenize("data/geo-mx-2004_min.json",replace=True,showProgressEach=10000)

** Processing data/geo-mx-2004_min.json ...
2020-05-12 19:53:31.173356 :: 0.000172 - Processing item #0 
** Processed 1000 lines. Saved to /home/jovyan/public/B6_ProyectoIntegrador/data/geo-mx-2004_min.npy.


In [34]:
index_=Index().computePostingLists(tokens,showProgressEach=10000)
len(index_.postlists.keys())

** Counting TF & DF ...
2020-05-12 19:53:32.067760 :: 0.001594 - Processing item #0 
** Calculating TF-IDF...
2020-05-12 19:53:32.110040 :: 0.043874 - Processing item #0 
** Sorting posting lists...
2020-05-12 19:53:32.221722 :: 0.155556 - Processing item #0 


1858

In [35]:
ds=index_.o("covid coronavirus pandemia cuarentena",tokenize=True)
print(len(ds),ds[:5])
docs=tokens.getToks(ds)
print(len(docs),docs[:5])
tokens.getTws(ds)

33 [12, 39, 42, 104, 143]
33 [['escuch', 'enlaz', 'inform', 'pandemi', 'retransmision'], ['altur', 'cuarenten', 'cuestion', 'si', 'amig'], ['si', 'sacrific', 'coloni', 'culiac', 'cov'], ['dic', 'vay', 'acab', 'cov', 'da'], ['abril', 'mes', 'clav', 'tem', 'cov']]


[{'coordinates': {'type': 'Point', 'coordinates': [-91.7702579, 18.6509625]},
  'created_at': 'Wed Apr 01 06:00:09 +0000 2020',
  'favorite_count': 0,
  'id': 1245229446715539456,
  'lang': 'es',
  'place': {'url': 'https://api.twitter.com/1.1/geo/id/c180018bfd4c2953.json',
   'full_name': 'Carmen, Campeche',
   'attributes': {},
   'country_code': 'MX',
   'id': 'c180018bfd4c2953',
   'country': 'México',
   'name': 'Carmen',
   'bounding_box': {'type': 'Polygon',
    'coordinates': [[[-92.468889, 17.873531],
      [-92.468889, 19.032122],
      [-90.797686, 19.032122],
      [-90.797686, 17.873531]]]},
   'place_type': 'city'},
  'replay_count': 0,
  'retweet_count': 0,
  'text': '#QuédateEnCasa \n#FelizMiércoles\nEscucha #UnidosPorLasAudienciasCovid19 enlazados a \n@RadioEducacion para informarse  de la pandemia; y la retransmisión de los conciertos de la Orquesta Sinfónica Nacional en honor a Beethoven en la programación del 1 de #abril de 2020🐬 https://t.co/EBgx6FdsOY',
  'timesta

In [7]:
gc.collect()
import sys
local_vars = list(locals().items())
tot=0
vars_=[]
for var, obj in local_vars:
    mem=sys.getsizeof(obj)
    tot+=mem
    vars_.append((mem,var))
print("Total =",tot)
vars_.sort(reverse=True)
print(vars_)

20

Total = 21002
[(5688, 'ds'), (5504, 'docs'), (1064, 'Tokens'), (1064, 'InteractiveShell'), (1064, 'Index'), (1064, 'FastText'), (1064, 'Counter'), (400, 'defaultdict'), (278, '_i7'), (248, '_oh'), (248, 'Out'), (195, '_i1'), (172, '_i6'), (172, '_i'), (167, '_i2'), (144, 'tweet_iterator'), (138, '_iii'), (138, '_i4'), (136, '_ih'), (136, 'In'), (133, '_ii'), (133, '_i5'), (113, '__doc__'), (88, 'tb'), (88, 'sys'), (88, 'pd'), (88, 'os'), (88, 'np'), (88, 'gc'), (88, 'datetime'), (88, '__builtins__'), (88, '__builtin__'), (80, '_dh'), (72, 'get_ipython'), (72, '_i3'), (64, 'tokens'), (64, 'quit'), (64, 'index_'), (64, 'exit'), (57, '__name__'), (53, '___'), (53, '__'), (28, '_7'), (28, '_'), (16, '__spec__'), (16, '__package__'), (16, '__loader__')]
